In [1]:
import pandas as pd
import re

In [17]:
df = pd.read_csv('aln_WT_kmer.sam',skiprows = 2,delim_whitespace=True,header=None, index_col=None,names =  ['QNAME','FLAG','RNAME','POS','MAPQ','CIGAR','RNEXT','PNEXT','TLEN','SEQ','QUAL','M1','M2','M3','M4','M5','M6','M7','M8','M9'])
#df.to_csv('1A_sam_output.csv', index = False)
df = df.iloc[:, : 10]
df['protospacer'] = 'Null'
df

,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,protospacer
0,A00351:76:HCLKMDSXX:3:1101:30101:1423_1:N:0:0A...,0,Chr3_GTL1_protospacer_region,1417,37,25M,*,0,0,GGTCACTCCCATTCTAGCTCAGCTA,Null
1,A00351:76:HCLKMDSXX:3:1101:30101:1423_1:N:0:0A...,0,Chr3_GTL1_protospacer_region,1422,37,25M,*,0,0,CTCCCATTCTAGCTCAGCTAAGCTT,Null
2,A00351:76:HCLKMDSXX:3:1101:30101:1423_1:N:0:0A...,0,Chr3_GTL1_protospacer_region,1427,37,25M,*,0,0,ATTCTAGCTCAGCTAAGCTTGATTA,Null
3,A00351:76:HCLKMDSXX:3:1101:30101:1423_1:N:0:0A...,0,Chr3_GTL1_protospacer_region,1432,37,25M,*,0,0,AGCTCAGCTAAGCTTGATTAGGTAC,Null
4,A00351:76:HCLKMDSXX:3:1101:30101:1423_1:N:0:0A...,0,Chr3_GTL1_protospacer_region,1437,37,25M,*,0,0,AGCTAAGCTTGATTAGGTACTAGTG,Null
...,...,...,...,...,...,...,...,...,...,...,...
147078,A00351:76:HCLKMDSXX:3:2678:13060:35258_4:N:0:0...,0,Chr3_GTL1_protospacer_region,1041,37,25M,*,0,0,TTCACTGTAAAATCACAAGGGGGAG,Null
147079,A00351:76:HCLKMDSXX:3:2678:13060:35258_4:N:0:0...,0,Chr3_GTL1_protospacer_region,1046,37,25M,*,0,0,TGTAAAATCACAAGGGGGAGGCATT,Null
147080,A00351:76:HCLKMDSXX:3:2678:13060:35258_4:N:0:0...,0,Chr3_GTL1_protospacer_region,1051,37,25M,*,0,0,AATCACAAGGGGGAGGCATTGCAAT,Null
147081,A00351:76:HCLKMDSXX:3:2678:13060:35258_4:N:0:0...,0,Chr3_GTL1_protospacer_region,1056,37,25M,*,0,0,CAAGGGGGAGGCATTGCAATGCAAC,Null


In [18]:
#check unique indel types
unique_labels = list(df.CIGAR.unique())
print(len(unique_labels))
def unwanted_labels(element):
    pattern = r"(\d+)([a-zA-Z]+)"
    match = re.search(pattern, element)
    if match:
        m = re.findall(pattern,element)
        if len(m) >1 :
            #print("Full match: % s" % (match.group(0)),m) 
            return element
        else:
            return 'wt'
    else:
        return False
unique_labels

46


['25M',
 '*',
 '18M1I6M',
 '13M1I11M',
 '8M1I16M',
 '3M1I21M',
 '6M1I18M',
 '11M1I13M',
 '16M1I8M',
 '8M1D17M',
 '13M1D12M',
 '17M1D8M',
 '12M1D13M',
 '7M1D18M',
 '5M1I19M',
 '10M1I14M',
 '15M1I9M',
 '20M1I4M',
 '9M1I15M',
 '14M1I10M',
 '19M1I5M',
 '6M1D19M',
 '11M1D14M',
 '16M1D9M',
 '18M1D7M',
 '3M1D22M',
 '4M1I20M',
 '5M2D20M',
 '10M2D15M',
 '4M1D21M',
 '17M1I7M',
 '12M1I12M',
 '7M1I17M',
 '19M1D6M',
 '14M1D11M',
 '9M1D16M',
 '8M2D17M',
 '2M1I22M',
 '21M1D4M',
 '7M2D18M',
 '10M1D15M',
 '15M1D10M',
 '20M1D5M',
 '11M2D14M',
 '5M1D20M',
 '9M2D16M']

In [19]:
wt_bank = {'p1':"CACAACGACG",'p2':"GCTACGTACG",'p3':"CGGGGAATCT",'p4':"GGTGGGCCCG",'p5':'CGCGAACGCG','p6':'CCCACGTTGC','p7':'CACATGTTTT'}


In [20]:
#protospacer PAM index (not python index, followed sam file index)
perfect_match_requirement = 25
p_bank = {'p1':238,'p2':744,'p3':941,'p4':1277,'p5':1366,'p6':1402,'p7':1497}
#+1 to make sure PAM is covered
acceptable_index_range = {key: {key+'start': val-perfect_match_requirement+1, key+'end': val-1}  for key, val in p_bank.items()}
acceptable_index_range

{'p1': {'p1start': 214, 'p1end': 237},
 'p2': {'p2start': 720, 'p2end': 743},
 'p3': {'p3start': 917, 'p3end': 940},
 'p4': {'p4start': 1253, 'p4end': 1276},
 'p5': {'p5start': 1342, 'p5end': 1365},
 'p6': {'p6start': 1378, 'p6end': 1401},
 'p7': {'p7start': 1473, 'p7end': 1496}}

In [21]:
#taken 10 bp
specific_index_range = {key: val-5 for key, val in p_bank.items()}
specific_index_range

{'p1': 233,
 'p2': 739,
 'p3': 936,
 'p4': 1272,
 'p5': 1361,
 'p6': 1397,
 'p7': 1492}

In [22]:
# from the start POS, find which protospacer the seq looked at

def protospacer_finder(x, range_dict):
    find = False
    for key,value_ in range_dict.items():
        #print(value_.values(), type(value_.values()))
        range_to_find = list(value_.values())
        #print(range_to_find)
        if x in range(range_to_find[0], range_to_find[1]):
            find = True
            return key
        else:
            pass
    if not find:
        #target seq not at protospacer region
        return find
#test
a = protospacer_finder(1368,acceptable_index_range) 
a

False

In [23]:
#select rows that may contain possible indels surrounding PAM
possible_indel_rows = df.loc[df['CIGAR'].apply(unwanted_labels) != False]
possible_indel_rows = possible_indel_rows.loc[possible_indel_rows['POS'].apply(protospacer_finder,args=(acceptable_index_range,)) != False]
possible_indel_rows.reset_index(drop=True, inplace=True)
possible_indel_rows['real_pam_seq'] = 'Null'
possible_indel_rows

,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,protospacer,real_pam_seq
0,A00351:76:HCLKMDSXX:3:1101:30101:1423_1:N:0:0A...,0,Chr3_GTL1_protospacer_region,1477,37,25M,*,0,0,ATGCATGCATGCATCCACATGTTTT,Null,Null
1,A00351:76:HCLKMDSXX:3:1101:30101:1423_1:N:0:0A...,0,Chr3_GTL1_protospacer_region,1482,37,25M,*,0,0,TGCATGCATCCACATGTTTTTGATT,Null,Null
2,A00351:76:HCLKMDSXX:3:1101:30101:1423_1:N:0:0A...,0,Chr3_GTL1_protospacer_region,1487,37,25M,*,0,0,GCATCCACATGTTTTTGATTCCTGA,Null,Null
3,A00351:76:HCLKMDSXX:3:1101:30101:1423_1:N:0:0A...,0,Chr3_GTL1_protospacer_region,1492,37,25M,*,0,0,CACATGTTTTTGATTCCTGAACTTA,Null,Null
4,A00351:76:HCLKMDSXX:3:1101:30101:1423_4:N:0:0C...,16,Chr3_GTL1_protospacer_region,1491,37,25M,*,0,0,CCACATGTTTTTGATTCCTGAACTT,Null,Null
...,...,...,...,...,...,...,...,...,...,...,...,...
17028,A00351:76:HCLKMDSXX:3:2678:13060:35258_1:N:0:0...,16,Chr3_GTL1_protospacer_region,1273,37,25M,*,0,0,GTGGGCCCGGCAGGGCAGGGGTAGA,Null,Null
17029,A00351:76:HCLKMDSXX:3:2678:13060:35258_1:N:0:0...,16,Chr3_GTL1_protospacer_region,1268,37,25M,*,0,0,GTGTGGTGGGCCCGGCAGGGCAGGG,Null,Null
17030,A00351:76:HCLKMDSXX:3:2678:13060:35258_1:N:0:0...,16,Chr3_GTL1_protospacer_region,1263,37,25M,*,0,0,AGCCTGTGTGGTGGGCCCGGCAGGG,Null,Null
17031,A00351:76:HCLKMDSXX:3:2678:13060:35258_1:N:0:0...,16,Chr3_GTL1_protospacer_region,1258,37,25M,*,0,0,GTAACAGCCTGTGTGGTGGGCCCGG,Null,Null


In [24]:
for i in range(possible_indel_rows.shape[0]):
    finder_result = protospacer_finder(possible_indel_rows.iloc[i]['POS'],acceptable_index_range)
    assert_if_wt = unwanted_labels(possible_indel_rows.iloc[i]['CIGAR'])
    possible_indel_rows.at[i,'protospacer'] = finder_result
    possible_indel_rows.at[i,'CIGAR'] = assert_if_wt
    #taken the seq from the segments
    starting_PAM_index = specific_index_range[finder_result] - possible_indel_rows.iloc[i]['POS']
    if starting_PAM_index >= 0 and possible_indel_rows.iloc[i]['POS']+starting_PAM_index+10 < possible_indel_rows.iloc[i]['POS']+25:
        possible_indel_rows.at[i,'real_pam_seq'] = possible_indel_rows.iloc[i]['SEQ'][starting_PAM_index:starting_PAM_index+10]
        if assert_if_wt == 'wt':
            if possible_indel_rows.loc[i,'real_pam_seq'] != wt_bank[possible_indel_rows.loc[i,'protospacer']]:
                possible_indel_rows.at[i,'real_pam_seq'] = 'Null'
            else:
                possible_indel_rows.at[i,'real_pam_seq'] = 'wt'
possible_indel_rows

,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,protospacer,real_pam_seq
0,A00351:76:HCLKMDSXX:3:1101:30101:1423_1:N:0:0A...,0,Chr3_GTL1_protospacer_region,1477,37,wt,*,0,0,ATGCATGCATGCATCCACATGTTTT,p7,Null
1,A00351:76:HCLKMDSXX:3:1101:30101:1423_1:N:0:0A...,0,Chr3_GTL1_protospacer_region,1482,37,wt,*,0,0,TGCATGCATCCACATGTTTTTGATT,p7,wt
2,A00351:76:HCLKMDSXX:3:1101:30101:1423_1:N:0:0A...,0,Chr3_GTL1_protospacer_region,1487,37,wt,*,0,0,GCATCCACATGTTTTTGATTCCTGA,p7,wt
3,A00351:76:HCLKMDSXX:3:1101:30101:1423_1:N:0:0A...,0,Chr3_GTL1_protospacer_region,1492,37,wt,*,0,0,CACATGTTTTTGATTCCTGAACTTA,p7,wt
4,A00351:76:HCLKMDSXX:3:1101:30101:1423_4:N:0:0C...,16,Chr3_GTL1_protospacer_region,1491,37,wt,*,0,0,CCACATGTTTTTGATTCCTGAACTT,p7,wt
...,...,...,...,...,...,...,...,...,...,...,...,...
17028,A00351:76:HCLKMDSXX:3:2678:13060:35258_1:N:0:0...,16,Chr3_GTL1_protospacer_region,1273,37,wt,*,0,0,GTGGGCCCGGCAGGGCAGGGGTAGA,p4,Null
17029,A00351:76:HCLKMDSXX:3:2678:13060:35258_1:N:0:0...,16,Chr3_GTL1_protospacer_region,1268,37,wt,*,0,0,GTGTGGTGGGCCCGGCAGGGCAGGG,p4,wt
17030,A00351:76:HCLKMDSXX:3:2678:13060:35258_1:N:0:0...,16,Chr3_GTL1_protospacer_region,1263,37,wt,*,0,0,AGCCTGTGTGGTGGGCCCGGCAGGG,p4,wt
17031,A00351:76:HCLKMDSXX:3:2678:13060:35258_1:N:0:0...,16,Chr3_GTL1_protospacer_region,1258,37,wt,*,0,0,GTAACAGCCTGTGTGGTGGGCCCGG,p4,wt


In [25]:
a = possible_indel_rows.loc[possible_indel_rows['protospacer'] == 'p5']
a.loc[a['real_pam_seq'] == 'wt']

,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,protospacer,real_pam_seq
27,A00351:76:HCLKMDSXX:3:1102:29288:20024_4:N:0:0...,16,Chr3_GTL1_protospacer_region,1350,37,wt,*,0,0,GTGAGTATAAACGCGAACGCGGACC,p5,wt
35,A00351:76:HCLKMDSXX:3:1103:27606:5181_1:N:0:0A...,0,Chr3_GTL1_protospacer_region,1350,37,wt,*,0,0,GTGAGTATAAACGCGAACGCGGACC,p5,wt
36,A00351:76:HCLKMDSXX:3:1103:27606:5181_1:N:0:0A...,0,Chr3_GTL1_protospacer_region,1355,37,wt,*,0,0,TATAAACGCGAACGCGGACCCCCTC,p5,wt
37,A00351:76:HCLKMDSXX:3:1103:27606:5181_1:N:0:0A...,0,Chr3_GTL1_protospacer_region,1360,37,wt,*,0,0,ACGCGAACGCGGACCCCCTCTCGCT,p5,wt
51,A00351:76:HCLKMDSXX:3:1104:17779:22858_1:N:0:0...,16,Chr3_GTL1_protospacer_region,1361,37,wt,*,0,0,CGCGAACGCGGACCCCCTCTCGCTA,p5,wt
...,...,...,...,...,...,...,...,...,...,...,...,...
16921,A00351:76:HCLKMDSXX:3:2671:20889:3129_4:N:0:0C...,16,Chr3_GTL1_protospacer_region,1353,37,wt,*,0,0,AGTATAAACGCGAACGCGGACCCCC,p5,wt
16922,A00351:76:HCLKMDSXX:3:2671:20889:3129_4:N:0:0C...,16,Chr3_GTL1_protospacer_region,1348,37,wt,*,0,0,GAGTGAGTATAAACGCGAACGCGGA,p5,wt
16965,A00351:76:HCLKMDSXX:3:2673:7898:12727_1:N:0:0A...,16,Chr3_GTL1_protospacer_region,1361,37,wt,*,0,0,CGCGAACGCGGACCCCCTCTCGCTA,p5,wt
16966,A00351:76:HCLKMDSXX:3:2673:7898:12727_1:N:0:0A...,16,Chr3_GTL1_protospacer_region,1356,37,wt,*,0,0,ATAAACGCGAACGCGGACCCCCTCT,p5,wt


counts_for_types = possible_indel_rows['real_pam_seq'].value_counts()
print(counts_for_types, type(counts_for_types))

In [83]:
leaf_name = '1A'
indel_count_series = possible_indel_rows.groupby(['protospacer', 'real_pam_seq']).size()
new_df = indel_count_series.to_frame(name = leaf_name).reset_index()
new_df = new_df[new_df.real_pam_seq != 'Null']
new_df

,protospacer,real_pam_seq,1A
0,p1,AACAAACGAC,6
1,p1,CACAAACGAA,6
2,p1,CACAAACGAC,3027
3,p1,CACAAATCGA,4
4,p1,CACAACCGAC,3
5,p1,CACAATCGAC,3
6,p1,CACACACGAC,3
7,p1,CACAGACGAC,6
8,p1,CAGACGACGA,1
9,p1,CCCCAACGAC,1


In [68]:
possible_indel_rows.protospacer.unique()

array(['p1', 'p2', 'p7', 'p4'], dtype=object)

In [87]:
new_df.to_csv('1A_indel_count.csv', index = False)